* Purpose: Script to extract editions information from worldcat given oclc_number
* License: GPLv3 (Free Software) 
* Date: Oct 15, 2018

In [7]:
# Load required libraries
from urllib.request import urlopen
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [8]:
# Finds how many total number of editions are for a given book
# Based on Oct 15, 2018 OCLC editions page structure
def find_total_editions_count(page_url):
    page = urlopen(page_url)
    page_html = BeautifulSoup(page, "html.parser")

    table_results_info = page_html.findAll("table",{"class":"tableResultsInfo"})
    count_info = table_results_info[0].findAll("strong")
    items_count = int(count_info[1].text)
    return items_count

In [9]:
# Finds how many total number of pages contain edition info for a given book
# Based on Oct 15, 2018 OCLC editions page structure
def get_editions_page_count(oclc_number):
    page_url = "https://www.worldcat.org/oclc/" + str(oclc_number) + "/editions"
    items_count = find_total_editions_count(page_url)
    page_count = round(items_count / 10)
    return page_count

In [10]:
# Gets a page of editions info and extracts required info
def get_edition_info(oclc_number, start_edition, edition_info_df):
    page_url = "https://www.worldcat.org/oclc/" + str(oclc_number) + "/editions?start_edition=" + str(start_edition)
    page = urlopen(page_url)
    page_html = BeautifulSoup(page, "html.parser")
    
    table_results = page_html.findAll("table", {"class": "table-results"})
    result_nodes = table_results[0].findAll("td", attrs={"class": "result"})
    
    for td_node in result_nodes:
        title_link = td_node.find('a', href=True)
        title = title_link.text.strip()
        
        author_div = td_node.find('div', attrs={"class": "author"})
        author = ""
        if author_div is not None:
            author = author_div.text.strip()
        
        book_format_td = td_node.findNext('td')
        book_format = ""
        if book_format_td is not None:
            book_format = book_format_td.text.strip()
        
        language_td = book_format_td.findNext('td')
        language = ""
        if language_td is not None:
            language = language_td.text.strip()
        
        date_td = language_td.findNext('td')
        date = ""
        if date_td is not None:
            date = date_td.text.strip()
        
        publisher_td = date_td.findNext('td')
        publisher = ""
        if publisher_td is not None:
            publisher = publisher_td.text.strip()

        edition_info_df = edition_info_df.append({'oclc_number':oclc_number, 'title': title, 'author': author, 'format': book_format, 'language': language, 'date':date, 'publisher': publisher}, ignore_index=True)
    return edition_info_df

In [11]:
# Read input info, i.e oclc_number
book_info_df = pd.read_csv("data/source/sociology_oclc_info.csv") 
book_info_df.head()

,BOOK_ID,ED_ID,oclc_number,Textbook,Book_Authors,Publisher,Year of Publication,STRUCTURE,INTRO,CH.TITLE_1,...,CH.AUTHORS_39,CH.TITLE_40,CH.LENGTH_40,CH.AUTHORS_40,CH.TITLE_41,CH.LENGTH_41,CH.AUTHORS_41,CH.TITLE_42,CH.LENGTH_42,CH.AUTHORS_42
0,296,296.1,824231,New directions in sociological theory,Paul Filmer; Michael Phillipson;,Collier Macmillan,1972,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,411,411.1,69205232,The Decomposition of Sociology,Irving Horowitz,Oxford University Press,1993,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,166,166.1,152432104,The social theories of LT Hobhouse,Hugh SeiverCarter,University of North Carolina Press,1927,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,447,447.1,270808917,"Collected papers Vol. 2, Studies in social the...",Alfred Schutz,M Nijhoff,1964,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,283,283.1,906380481,Sociological Theory what went wrong diagnosis ...,Nicos Mouzelis,Routledge,1995,Topics; Paradigms,The book focuses both on the past and the futu...,Introduction,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Loop through the oclc numbers, get the edition info, and store that into a panada's dataframe
edition_info_df = pd.DataFrame()

for index, row in book_info_df.iterrows():
    oclc_number = row["oclc_number"]
    page_count = get_editions_page_count(oclc_number)
    
    for page_number in range(page_count):
        start_edition = (page_number * 10) + 1
        edition_info_df = get_edition_info(oclc_number, start_edition, edition_info_df)    

In [13]:
# Write the result set dataframe into a file
edition_info_df.to_csv("data/extracted/edition_info.csv", header=True, columns=["oclc_number","title","author","format","language","date","publisher"])